In [1]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [4]:
esp_ff = foyer.Forcefield(forcefield_files="/Users/madilyn/Projects/repos/forcefields/xml_files/PIDTBT_nC16.xml")

# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
molecule = mb.load("/Users/madilyn/Projects/repos/forcefields/PIDTBT_nC16_typed.mol2")

for p in molecule.particles():
    p.name = f"_{p.name}"

box = mb.fill_box(compound=molecule, n_compounds=1, box=[10,10,10])
box_pmd = esp_ff.apply(box)

snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)




#setting cpu and simulation
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

#setting the integrator
kt = 1.2
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt = 0.0001)
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=1.2, tau=0.5) #what is tau?
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_CPDT.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)



/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C37
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C31
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C51
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H42
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H38
  warnings.warn(
/Users/madilyn/miniconda3/envs/espa

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C0 pos=([ 0.5621 -0.7596  0.2153]), 4 bonds, id: 6225296544> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C1 pos=([ 0.4517 -0.6817  0.1282]), 3 bonds, id: 6222681520> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C2 pos=([ 0.4903 -0.5295  0.1106]), 3 bonds, id: 6225296496> does not have 

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:455: UserWarning: Parameters have been assigned to all proper dihedrals.  However, there are more parameterized dihedrals (4188) than total system dihedrals (698).  This may be due to having multiple periodic dihedrals for a single system dihedral.
  warnings.warn(msg)
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 284, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:926: UserWarning: Parametrized structure has non-zero charge.Structure's total charge: 0.00046515464782714844
  warnings.warn(
*Warning*: Systems with many particle types perform poorly or 

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [6]:
sim.run(1e4,write_at_start=True)

**ERROR**: Particle with unique tag 124 has NaN for its position.


RuntimeError: Error computing cell list